In [1]:
import os, glob, sys
import pandas as pd
import geopandas as gpd
import fiona
from shapely.geometry import Polygon, mapping
import matplotlib.pyplot as plt
import folium
import rasterio
import rasterio.mask
import rasterio.plot

sys.path.insert(0, '/export/miro/ndeffense/LBRAT2104/GIT/eo-toolbox/tools/')

import folium_def


# Input / Output

## Input

In [38]:
path = '/export/projects/FAO-EOStat-Senegal/'

odk_crop_csv     = f'{path}ODK/FormSenegal05.csv'
odk_non_crop_csv = f'{path}ODK/FormSenegal07.csv'

odk_shp = f'{path}ODK/polygons_ODK.shp'
roi_shp = f'{path}ROI/Senegal_DepartementNioro.shp'

lut_crop_csv     = f'{path}LUT/LUT_crop.csv'
lut_non_crop_csv = f'{path}LUT/LUT_non_crop.csv'

## Output

In [32]:
gps_shp_output = f'{path}Output_Nico/gps_with_data.shp'

odk_crop_shp_output = f'{path}Output_Nico/odk_crop_with_data.shp'

odk_non_crop_shp_output = f'{path}Output_Nico/odk_non_crop_with_data.shp'

map_html_output = f'{path}Output_Nico/odk_gps_with_data.html'


# Open Look-up-Table

In [46]:
lut_crop_df = pd.read_csv(lut_crop_csv)

lut_crop_df.head()



,crop_code,crop_name
0,1,Arachide
1,2,Aubergine
2,3,Béréf
3,4,Bissap
4,5,Coton


In [45]:
lut_non_crop_df = pd.read_csv(lut_non_crop_csv)

lut_non_crop_df.head()

,lc_code,lc_name
0,1,Prairie
1,2,Jachère
2,3,Broussailes
3,4,Forêt
4,5,Sol nu


# GPX files

In [5]:

dict_gpx_list = []

wrong_pid = 0

for gpx_file in glob.glob(f'{path}GPS/*/*.gpx'):

    filename = os.path.basename(gpx_file)

    
    pid = filename[filename.find("Piste")+5:-4].lstrip('_ ')
    pid = pid.replace(' ','-')
    pid = pid.replace('---','-')
    pid = pid.replace('--','-')
    pid = pid.split('-')

    
    if len(pid) <= 5:
        pid = ''.join(pid)

        layer = fiona.open(gpx_file, layer='tracks')

        crs = layer.crs

        if crs['init'] == 'epsg:4326':

            dict_gpx = {'filename': filename,
                        'id': pid,
                        'coordinates': layer[0]['geometry']['coordinates'][0]}

            dict_gpx_list.append(dict_gpx)
    else:
        wrong_pid += 1

print(f'Good PID : {len(dict_gpx_list)}')
print(f'Wrong PID : {wrong_pid}')


Good PID : 838
Wrong PID : 36


## Store GPX tracks in GeoDataFrame

- Apply "zero" buffer to clean geometries
- Drop duplicates

In [6]:
df = pd.DataFrame.from_dict(dict_gpx_list)

df['geometry'] = df.coordinates.apply(Polygon)

df = df.drop('coordinates', axis=1)

gps_gdf = gpd.GeoDataFrame(df, crs='epsg:4326', geometry='geometry')

gps_gdf = gps_gdf.drop_duplicates()

gps_gdf['geometry'] = gps_gdf.buffer(0)

gps_gdf = gps_gdf.to_crs(epsg=32628)

gps_gdf['area'] = gps_gdf['geometry'].area.round(2)

gps_gdf = gps_gdf.to_crs(epsg=4326)


print(f'There are {len(gps_gdf)} polygons obtained with the GPS')
gps_gdf.head()

There are 350 polygons obtained with the GPS


,filename,id,geometry,area
0,Piste_6220102-2-4-1-1.gpx,62201022411,"POLYGON ((-15.65158 13.71486, -15.65157 13.714...",8842.45
1,Piste_6220102-2-4-1-2.gpx,62201022412,"POLYGON ((-15.64866 13.71411, -15.64868 13.714...",17477.66
2,Piste_6220102-2-4-1-3.gpx,62201022413,"POLYGON ((-15.64766 13.71621, -15.64765 13.716...",9808.82
3,Piste_6220102-2-4-1-4.gpx,62201022414,"POLYGON ((-15.64763 13.71488, -15.64762 13.714...",5276.49
4,Piste_6220102-2-16-4.gpx,62201022164,"POLYGON ((-15.64915 13.70645, -15.64915 13.706...",4267.04


# ODK

## Load cropland data from ODK

- Create ID based on fields

In [8]:
odk_crop_df = pd.read_csv(odk_crop_csv)

odk_crop_df['id'] = odk_crop_df['id-Commune'].astype(str) + odk_crop_df['id-District'].astype(str) + odk_crop_df['id-Concession'].astype(str) + odk_crop_df['id-Menage'].astype(str) + odk_crop_df['id-Parcelle'].astype(str)

odk_crop_df = odk_crop_df[['id','Informations_champ-Melange','Informations_champ-Culture','Informations_champ-Surface']]

odk_crop_df['Informations_champ-Surface'] = odk_crop_df['Informations_champ-Surface'].round(2)


odk_crop_df = odk_crop_df.merge(lut_crop_df, left_on='Informations_champ-Culture', right_on='crop_code')


odk_crop_df = odk_crop_df.drop_duplicates(subset=['id'])

print(f'There are {len(odk_crop_df)} rows in the ODK csv.')

odk_crop_df



There are 231 rows in the ODK csv.


,id,Informations_champ-Melange,Informations_champ-Culture,Informations_champ-Surface,crop_code,crop_name
0,6220201174341,1,1.0,31297.99,1,Arachide
1,6220201171611,1,1.0,17649.58,1,Arachide
2,6220201171631,1,1.0,10117.53,1,Arachide
3,6220201171661,1,1.0,7216.51,1,Arachide
4,6220201173221,1,1.0,7180.45,1,Arachide
...,...,...,...,...,...,...
230,62201022539103,1,9.0,8774.09,9,Maïs
231,6220205512446,1,9.0,12192.00,9,Maïs
232,6220303428444,1,9.0,78.58,9,Maïs
233,6220102321131,1,8.0,358.62,8,Gombo


## Load cropland polygons from ODK

- Convert Linstring to Polygon
- Drop duplicates
- Apply "zero" buffer to clean geometries

In [9]:
odk_gdf = gpd.read_file(odk_shp)

odk_gdf['geometry'] = [Polygon(mapping(x)['coordinates']) for x in odk_gdf.geometry]

odk_gdf['id'] = odk_gdf['id-Commune'].astype(str) + odk_gdf['id-Distric'].astype(str) + odk_gdf['id-Concess'].astype(str) + odk_gdf['id-Menage'].astype(str) + odk_gdf['id-Parcell'].astype(str)

#odk_gdf = odk_gdf[['id','Informations_champ-Melange','Informations_champ-Culture','Informations_champ-Surface','geometry']]

odk_gdf['geometry'] = odk_gdf.buffer(0)


odk_gdf = odk_gdf[['id','geometry']]

odk_gdf = odk_gdf.to_crs(epsg=32628)

odk_gdf['area'] = odk_gdf['geometry'].area.round(2)

odk_gdf = odk_gdf.to_crs(epsg=4326)

odk_gdf = odk_gdf.drop_duplicates(subset=['id'])


print(f'There are {len(odk_gdf)} polygons obtained with the tablet (ODK)')

odk_gdf.head()

There are 232 polygons obtained with the tablet (ODK)


,id,geometry,area
0,6220201174341,"POLYGON ((-15.98145 13.82769, -15.98163 13.826...",31096.39
1,6220201173222,"POLYGON ((-15.97993 13.82258, -15.97946 13.822...",9556.25
2,6220201171611,"POLYGON ((-15.98027 13.83368, -15.97899 13.833...",17535.89
3,6220201171631,"POLYGON ((-15.98141 13.83367, -15.98150 13.833...",10052.36
4,6220201171661,"POLYGON ((-15.98147 13.83311, -15.98148 13.833...",7170.03


## Load non-cropland data from ODK

In [50]:
odk_non_crop_df = pd.read_csv(odk_non_crop_csv)

odk_non_crop_df = odk_non_crop_df[['Picture-LC_Class','Picture-Other_Non_Cropland','start-geopoint-Latitude','start-geopoint-Longitude']]

odk_non_crop_df = odk_non_crop_df.rename(columns={"start-geopoint-Longitude": "longitude", "start-geopoint-Latitude": "latitude"})

odk_non_crop_df = odk_non_crop_df.merge(lut_non_crop_df, left_on='Picture-LC_Class', right_on='lc_code')

odk_non_crop_gdf = gpd.GeoDataFrame(odk_non_crop_df, geometry=gpd.points_from_xy(odk_non_crop_df.longitude, odk_non_crop_df.latitude, crs="EPSG:4326"))

odk_non_crop_gdf = odk_non_crop_gdf.drop(columns=['longitude', 'latitude'])

odk_non_crop_gdf = odk_non_crop_gdf.to_crs(epsg=32628)

odk_non_crop_gdf['geometry'] = odk_non_crop_gdf.buffer(10, resolution=24)

odk_non_crop_gdf = odk_non_crop_gdf.to_crs(epsg=4326)


odk_non_crop_gdf

,Picture-LC_Class,Picture-Other_Non_Cropland,lc_code,lc_name,geometry
0,5,NaN,5,Sol nu,"POLYGON ((-15.97416 13.81919, -15.97416 13.819..."
1,5,NaN,5,Sol nu,"POLYGON ((-15.98149 13.83366, -15.98149 13.833..."
2,5,NaN,5,Sol nu,"POLYGON ((-15.97814 13.83422, -15.97814 13.834..."
3,5,NaN,5,Sol nu,"POLYGON ((-15.55357 13.76242, -15.55357 13.762..."
4,5,NaN,5,Sol nu,"POLYGON ((-15.45145 13.76356, -15.45145 13.763..."
...,...,...,...,...,...
75,2,NaN,2,Jachère,"POLYGON ((-15.76676 13.74416, -15.76676 13.744..."
76,2,NaN,2,Jachère,"POLYGON ((4.62108 50.66616, 4.62108 50.66616, ..."
77,2,NaN,2,Jachère,"POLYGON ((4.62104 50.66613, 4.62104 50.66613, ..."
78,2,NaN,2,Jachère,"POLYGON ((-16.43940 16.04345, -16.43940 16.043..."


# Merge ODK data with ODK geometries

In [33]:
odk_with_data_gdf = odk_gdf.merge(odk_crop_df, on='id')

print(f'There are {len(odk_with_data_gdf)} polygons obtained with the tablet (ODK)')

odk_with_data_gdf.head()

There are 231 polygons obtained with the tablet (ODK)


,id,geometry,area,Informations_champ-Melange,Informations_champ-Culture,Informations_champ-Surface,crop_code,crop_name
0,6220201174341,"POLYGON ((-15.98145 13.82769, -15.98163 13.826...",31096.39,1,1.0,31297.99,1,Arachide
1,6220201173222,"POLYGON ((-15.97993 13.82258, -15.97946 13.822...",9556.25,1,13.0,9618.22,13,Pastèque
2,6220201171611,"POLYGON ((-15.98027 13.83368, -15.97899 13.833...",17535.89,1,1.0,17649.58,1,Arachide
3,6220201171631,"POLYGON ((-15.98141 13.83367, -15.98150 13.833...",10052.36,1,1.0,10117.53,1,Arachide
4,6220201171661,"POLYGON ((-15.98147 13.83311, -15.98148 13.833...",7170.03,1,1.0,7216.51,1,Arachide


# Merge ODK data with GPS geometries

In [34]:
gps_with_data_gdf = gps_gdf.merge(odk_crop_df, on='id', how='left')

print(f'There are {len(gps_with_data_gdf)} polygons obtained with GPS')

gps_with_data_gdf.head()

There are 350 polygons obtained with GPS


,filename,id,geometry,area,Informations_champ-Melange,Informations_champ-Culture,Informations_champ-Surface,crop_code,crop_name
0,Piste_6220102-2-4-1-1.gpx,62201022411,"POLYGON ((-15.65158 13.71486, -15.65157 13.714...",8842.45,1.0,1.0,5266.50,1.0,Arachide
1,Piste_6220102-2-4-1-2.gpx,62201022412,"POLYGON ((-15.64866 13.71411, -15.64868 13.714...",17477.66,1.0,11.0,16867.89,11.0,Mil
2,Piste_6220102-2-4-1-3.gpx,62201022413,"POLYGON ((-15.64766 13.71621, -15.64765 13.716...",9808.82,1.0,11.0,6282.66,11.0,Mil
3,Piste_6220102-2-4-1-4.gpx,62201022414,"POLYGON ((-15.64763 13.71488, -15.64762 13.714...",5276.49,1.0,9.0,3493.89,9.0,Maïs
4,Piste_6220102-2-16-4.gpx,62201022164,"POLYGON ((-15.64915 13.70645, -15.64915 13.706...",4267.04,NaN,NaN,NaN,NaN,NaN


In [51]:
odk_with_data_gdf.to_file(filename=odk_crop_shp_output, driver='ESRI Shapefile')

odk_non_crop_gdf.to_file(filename=odk_non_crop_shp_output, driver='ESRI Shapefile')

gps_with_data_gdf.to_file(filename=gps_shp_output, driver='ESRI Shapefile')

# Interactive plot with `folium`

https://leafletjs.com/reference-1.6.0.html#path-option

https://python-visualization.github.io/folium/quickstart.html

https://geopandas.org/gallery/polygon_plotting_with_folium.html

https://bikeshbade.com.np/tutorials/Detail/?title=Beginner+guide+to+python+Folium+module+to+integrate+google+earth+engine&code=8

https://nbviewer.org/github/python-visualization/folium/blob/master/examples/ImageOverlay.ipynb

In [53]:
#f = folium.Figure(width=1000, height=700)

m = folium.Map(location = [13.743747099563299, -15.772308355932424], zoom_start=11.5)#.add_to(f)

basemap_dict = folium_def.get_basemap()

basemap_dict['Google Satellite'].add_to(m)

# Plot ROI

#roi_gdf = gpd.read_file(roi_shp)

#sim_geo = gpd.GeoSeries(roi_gdf['geometry']).simplify(tolerance=0.001)
#geo_j = sim_geo.to_json()
#geo_j = folium.GeoJson(data=geo_j,
#                       style_function=lambda x: {'fillOpacity': 0, 'color': 'black'})
#geo_j.add_to(m)

# Plot GPX polygons in red

for _, r in gps_with_data_gdf.iterrows():
    sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,
                           style_function=lambda x: {'fillOpacity': 0, 'color': 'red'})
    
    html = f'''<b>GPS</b><br>
    ID : {r['id']}<br>
    Crop type : {r['crop_code']} - {r['crop_name']}<br>
    Area : {r['area']}
    '''
    
    iframe = folium.IFrame(html, width=250, height=150)
    folium.Popup(iframe).add_to(geo_j)
    #folium.Popup('crop type ' + str(r['Informations_champ-Culture'])).add_to(geo_j)
    geo_j.add_to(m)

# Plot ODK cropland polygons
# --------------------------

for _, r in odk_with_data_gdf.iterrows():
    sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,
                           style_function=lambda x: {'fillOpacity': 0, 'color': 'magenta'})

    html = f'''<b>ODK - Cropland</b><br>
    ID : {r['id']}<br>
    Crop type : {r['crop_code']} - {r['crop_name']}<br>
    Area : {r['area']}<br>
    Surface ODK : {r['Informations_champ-Surface']}
    '''
    
    iframe = folium.IFrame(html, width=250, height=150)
    folium.Popup(iframe).add_to(geo_j)
    geo_j.add_to(m)

# Plot ODK non-cropland points
# ----------------------------

for _, r in odk_non_crop_gdf.iterrows():
    sim_geo = gpd.GeoSeries(r['geometry'])#.simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,
                           style_function=lambda x: {'fillOpacity': 0, 'color': 'blue'})

    html = f'''<b>ODK - Non cropland</b><br>
    Land Cover : {r['lc_code']} - {r['lc_name']}<br>
    Comment : {r['Picture-Other_Non_Cropland']}<br>
    '''
    
    iframe = folium.IFrame(html, width=250, height=150)
    folium.Popup(iframe).add_to(geo_j)
    geo_j.add_to(m)



m = folium_def.add_categorical_legend(m, 'Legend',
                             colors = ['red','magenta','blue'],
                             labels = ['GPX', 'ODK - cropland','ODK - non cropland'])


m.save(map_html_output)

m